<a href="https://colab.research.google.com/github/KrisSandy/InformationRetrieval/blob/master/Vector_Space_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Space Model

## Background


#### Models

There are different models available for Information Retrieval. Some of them are shown below


* **Boolean Model: ** In this model, terms in the document collection are represented by 1s and 0s indicating presence or absence of the term in a document. To retrieve informtion for a query string, boolean operations are done against each query term in the term-document matrix to select the documents that contain the query words.
* **Vector Space Model:** In this model, each document is represented as a vector in a n-dimentional space with each term representing a dimention. User queries are represented in the same way in the space. Similarity of the documents and the query are calculated to rank the documents in order of relavence to retrieve them.


#### Weighting Schemes

Weights are given to terms in the document to represent the importance of the term in the documents. Some of the weighting schemes are 



*   TF-IDF (Term Frequency - Inverse Document Frequency)
*   BM25/Okapi
* Pivoted Normalisation


In this assignment, I will be using Vector Space Model and TF-IDF weighting scheme to find and rank the document in terms of similarity.


## Document and Query Representation

Below are the document set and the query for which the documents should be ranked

#### Documents

D1: Shipment of gold damaged in a fire

D2: Delivery of silver arrived in a silver truck

D3: Shipment of gold arrived in a truck

#### Query 

Q:  gold silver truck.



### Calculate Weights

#### Pre processing

All the given documents are preprocessed by



1.   Converting the case to lower
2.   Removing stopwords


#### Term Frequency table and Inverse Document Frequency

> **TF** = Number of times term t appears in a document.

Term frequency of the given documents are as shown below

In [0]:
tf

,truck,gold,fire,delivery,silver,arrived,shipment,damaged
d1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
d2,1.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0
d3,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


#### Inverse Document Frequency

>** IDF ** = log(N/n)

> *where 'N' is total number of documents in the collection and 'n' is number of documents that contain term t*

For Example: IDF for term 'fire' is calculated as below

N = 3

n = 1 (as it appeared in 1 document).

**IDF = log(3/1) = 0.477**

Computing the IDF for rest of the terms in similar fashion, IDF table will be as shown below

In [0]:
idf

,truck,gold,fire,delivery,silver,arrived,shipment,damaged
idf,0.176091,0.176091,0.477121,0.477121,0.477121,0.176091,0.176091,0.477121


#### Normalising TF

Term Frequencies should be normalised because number of times a term occured doesn't truely increase the significance of the term to the times it occured.

I will be using sublinear scaling technique to normalise tf

$wf_{i,d} =
  \begin{cases}
    1+\log(tf_{i,d})       & \quad \text{if } tf_{i,d} > 0\\
    0  & \quad \text{otherwise}
  \end{cases}
$

For example normalised tf for term 'silver' in d2 is calculated as below 

**wj = 1+log(2) = 1.30**

In [0]:
tf_norm

,truck,gold,fire,delivery,silver,arrived,shipment,damaged
d1,0.0,1.0,1.0,0.0,0.00000,0.0,1.0,1.0
d2,1.0,0.0,0.0,1.0,1.30103,1.0,0.0,0.0
d3,1.0,1.0,0.0,0.0,0.00000,1.0,1.0,0.0


#### Weights

Weights of each term in the document is calculated by multiplying TF (normalized) and IDF

$w_{i,j} = TF*IDF = f_{i,j} * log(\frac{N}{n})$

$f_{i,j}$ is normalised term frequency

For Example, weight of term 'fire' in d1 is calculated as below

**w = 1 * 0.477 = 0.477** 

In [0]:
dw

,truck,gold,fire,delivery,silver,arrived,shipment,damaged
d1,0.000000,0.176091,0.477121,0.000000,0.000000,0.000000,0.176091,0.477121
d2,0.176091,0.000000,0.000000,0.477121,0.620749,0.176091,0.000000,0.000000
d3,0.176091,0.176091,0.000000,0.000000,0.000000,0.176091,0.176091,0.000000


#### Query Weights

For Query weights, less credence has been give to tf-idf and a default value of 0.5 is considered.

In [0]:
qw

,truck,gold,fire,delivery,silver,arrived,shipment,damaged
q,0.5,0.5,0.0,0.0,0.5,0.0,0.0,0.0


### Calculating Similarity

As the documents and query are represented as vectors in n-dimentional space, similarity of a document and a query is calculated by calculating the cosine of the angle between the document and the query

$sim(\overrightarrow{d_j}, \overrightarrow{q}) = \frac{d_j\bullet{q}}{|d_j||q|} = \frac{\displaystyle\sum_{i=1}^n(w_{i,j}*w_{i,q})}{\sqrt{\displaystyle\sum_{i=1}^{n}w_{i,j}^2}\sqrt{\displaystyle\sum_{i=1}^{n}w_{i,q}^2}}$

For Example similarity between document d1 and q is calculated as below

$sim(d1,q) = \frac{(0.176*0.5)}{\sqrt{0.176^2 + 0.176^2+0.477^2+0.477^2}\sqrt{0.5^2+0.5^2+0.5^2}} = 0.141$

In [0]:
print("Similarity between d1 and q is {}".format(similarity(qw.loc[q_row], dw.loc['d1'])))
print("Similarity between d2 and q is {}".format(similarity(qw.loc[q_row], dw.loc['d2'])))
print("Similarity between d3 and q is {}".format(similarity(qw.loc[q_row], dw.loc['d3'])))

Similarity between d1 and q is 0.14135252212346566
Similarity between d2 and q is 0.5599663010899988
Similarity between d3 and q is 0.5773502691896257


From the above similarity scores, the documents are ranked as d3, d2, d1.

The similarty scores of d2 and d3 also almost same. The result depends on the various factors like


1.   **Default Query Weights**: In this example, query weights has been taken as constant values. Some other variations include considering the idf value (e.g. $\alpha + \beta(idf)$ where $\alpha, \beta$ are the tuning parameters)
2.   **Normalisation**: Normalisation menthods used have slight impact on the document weights and hence the similarity score. In the example, using tf without normalisation will result in document d2 ranked higher than document d3.
3. **Sophisticated Weighting Schemes**: More sophisticated weighing schemes like BM25 will have impact on the document weights and hence on the similarity score 



## Similarity Calculation

As the document d1 is changed in each of the case, Term Frequency and weights needs to be recalculated. IDF will remain the same

Similarity score will need to be recalculated as well and the new similarity score between d1 and q is as below

#### a) D1 = Shipment of gold damaged in a fire. Fire.

In [0]:
d1 = "Shipment of gold damaged in a fire. Fire."
print(recalculate_similarity([d1,d2,d3], 'd1'))

0.12374520733918673


Similarity between the document d1 and query decreased from 0.141 to 0.123 by adding a non relavent term to the document d1.





#### b) D1 = Shipment of gold damaged in a fire. Fire. Fire.

In [0]:
d1 = "Shipment of gold damaged in a fire. Fire. Fire. "
print(recalculate_similarity([d1,d2,d3], 'd1'))

0.11464828710168476


Similarity between the document d1 and query further decreased from 0.141 to 0.114 by adding two non relavent terms to the document. As the number of non relavent terms increase in the document, the similary between the document and query will reduce.

#### c) D1 = Shipment of gold damaged in a fire. Gold.

In [0]:
d1 = "Shipment of gold damaged in a fire. Gold. "
print(recalculate_similarity([d1,d2,d3], 'd1'))

0.18020091957864967


Similarity between the document d1 and query increased  from 0.141 to 0.180  by adding a relavent term to the document.

#### d) D1 = Shipment of gold damaged in a fire. Gold. Gold.

In [0]:
d1 = "Shipment of gold damaged in a fire. Gold. Gold. "
print(recalculate_similarity([d1,d2,d3], 'd1'))

0.20176998483273476


Similarity between the document d1 and query further increased  from 0.141 to 0.201 by adding two relavent terms to the document. As the number of relavent terms increase in the document, the similary between the document and query will increase.

## Relevance Feedback

The performance of the IR system can be improved by considering the user feedback and modifying the query based on the feedback

### Rocchio Relevance Feedback

In this approach, if the user marks some of the documents retrieved as relevant (denoted by $D_r$) and some as non relavent (denoted by $D_n$), the query weights can be recalculated as below

$\overrightarrow{q} = \alpha{\overrightarrow{q}}+\frac{\beta}{|D_r|}\displaystyle\sum_{d_j \epsilon D_r}\overrightarrow{d_j} - \frac{\gamma}{|D_n|}\displaystyle\sum_{d_j \epsilon D_n}\overrightarrow{d_j}$

$\alpha, \beta, \gamma$  are the constants which can be tuned to increase/decrease the relavence of positive and negative feedback.

In this question, user has given a feedback that document 3 is relavent which means the relavent documents set $D_r$ contain d2. However the user has not explicitly marked any non relavent documents and hence the non relavent document set $D_n$ will be empty. 

Optimal values are $\alpha, \beta, \gamma$ are usually 1, 0.75 and 0.15 respectively and I have used the same for the calculations, but these can be changed based on the IR system requirements.

#### New Query Weights

In [0]:
qw_new = rocchio_rf(dw, qw, ['d3'], [])
qw_new

,truck,gold,fire,delivery,silver,arrived,shipment,damaged
q,0.632068,0.632068,0.0,0.0,0.5,0.132068,0.132068,0.0


From the recalculated query weights, weight of the term in query is increased by 0.75 times the weight of document d3. 

#### New Document Similarities

Document similarities are calculated based on new query weigts

In [0]:
print("Similarity of document d1 changed from {} to {}".format(similarity(qw.loc[q_row], dw.loc['d1']), similarity(qw_new.loc[q_row], dw.loc['d1'])))
print("Similarity of document d2 changed from {} to {}".format(similarity(qw.loc[q_row], dw.loc['d2']), similarity(qw_new.loc[q_row], dw.loc['d2'])))
print("Similarity of document d3 changed from {} to {}".format(similarity(qw.loc[q_row], dw.loc['d3']), similarity(qw_new.loc[q_row], dw.loc['d3'])))

Similarity of document d1 changed from 0.14135252212346566 to 0.1796965371785806
Similarity of document d2 changed from 0.5599663010899988 to 0.5201751026777683
Similarity of document d3 changed from 0.5773502691896257 to 0.7339652844812885


After considering the user feedback that document d3 is relavent, and applying rocchio's relavence feedback method, the similarity score of document d3 has increased considerably. Similarity score of document d1 increased as well because of similarity between document d1 and d3 (i.e. term 'gold' with increased query weight because of d3 occur in document d1).

## Appendix (Code)

In [0]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import pandas as pd
import numpy as np
import string
from math import log10, sqrt

nltk.download('stopwords');
nltk.download('punkt');

d1 = "Shipment of gold damaged in a fire"
d2 = "Delivery of silver arrived in a silver truck"
d3 = "Shipment of gold arrived in a truck"
coll = [d1, d2, d3]
q = "gold silver truck"

doc_list = ['d1', 'd2', 'd3']
idf_row = 'idf'
q_row = 'q'
stopwords_eng = stopwords.words('english') + list(string.punctuation)
unique_words = list(set([w.lower() for d in coll for w in nltk.word_tokenize(d) if w not in stopwords_eng]))

def get_tf(coll):
  tf = pd.DataFrame(0.0, columns=unique_words, index=doc_list)
  for i, d in enumerate(coll):
    for word in nltk.word_tokenize(d):
      if word.lower() in unique_words:
        tf.at[doc_list[i], word.lower()] += 1
  return tf

def get_idf(coll):  
  N = len(coll)
  idf = pd.DataFrame(columns=unique_words, index=[idf_row])
  for col in tf.columns:
    n = len([c for c in tf[col] if c > 0.0])
    idf.at[idf_row, col] = log10(N/n)
  return idf

def normalise_tf(tf):
  tf_norm = tf.copy()
  for row in tf.index:
    for col in tf.columns:
      if tf.at[row, col] > 0:
        tf_norm.at[row, col] = 1 + log10(tf.at[row, col])
  return tf_norm
      
def get_weights(tf, idf):
  dw = pd.DataFrame(0.0, columns=unique_words, index=doc_list)
  for row in doc_list:
    for col in unique_words:
      dw.at[row, col] = tf.at[row, col] * idf.at[idf_row, col]
  return dw
    
def get_query_weights(q, idf):
  q_words = [w for w in q.split(" ") if w not in stopwords_eng]
  qw = pd.DataFrame(0.0, columns=unique_words, index=[q_row])
  for word in q_words:
    qw.at[q_row, word] = 0.5
  return qw
  
def similarity(q, d):
  dot = np.dot(d, q)
  dl = np.linalg.norm(d)
  ql = np.linalg.norm(q)
  print(dot)
  print(dl)
  print(ql)
  return dot/(dl*ql)
  
def recalculate_similarity(coll, doc):
  tf = get_tf(coll)
  tf_norm = normalise_tf(tf)
  dw = get_weights(tf_norm, idf)
  return (similarity(qw.loc[q_row], dw.loc[doc]))
  
def rocchio_rf(dw, qw, dr, dn):
  alpha = 1
  beta = 0.75
  gamma = 0.15
  if len(dr) > 0:
    centroid_dr = np.mean(dw.loc[dr])
  else:
    centroid_dr = np.zeros(len(dw.columns))
  if len(dn) > 0:
    centroid_dn = np.mean(dw.loc[dn])
  else:
    centroid_dn = np.zeros(len(dw.columns))
  qw_new = alpha*qw.loc[q_row] + beta*centroid_dr - gamma*centroid_dn
  qw_new = pd.DataFrame(qw_new, columns=['q']).transpose()
  return qw_new
  
tf = get_tf(coll)
idf = get_idf(coll)
tf_norm = normalise_tf(tf)
dw = get_weights(tf_norm, idf)
qw = get_query_weights(q, idf)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Improvements

In classic Rocchio's relavence feedback algorithm, we modify the query weights using a set of relavent documents and non relavent documents identified by the user. If the user marks some of the documents retrieved as relevant (denoted by $D_r$) and some as non relavent (denoted by $D_n$), the query weights can be recalculated as below

$\overrightarrow{q} = \alpha{\overrightarrow{q}}+\frac{\beta}{|D_r|}\displaystyle\sum_{d_j \epsilon D_r}\overrightarrow{d_j} - \frac{\gamma}{|D_n|}\displaystyle\sum_{d_j \epsilon D_n}\overrightarrow{d_j}$

$\alpha, \beta, \gamma$  are the constants which can be tuned to increase/decrease the relavence of positive and negative feedback.




If the user instead of identifying the relavent and non relavent documents, user give a feedback ranging from 1 to 5 where 1 means very non relavent and 5 means very relavent, we can modify the Rocchio's algorithm to take this into account. Similar to classic algorithm, we can calculate the centroids of all the documents in each group and multiply it with a weight assigned to each group. These weights can vary from a range of -1 to 1. Documents in very non relevant category can be given a more negative weight and documents in very relavent category can be give more positive weight. Finally adding all these to query weights will give a query point placed closer to relavent documents  and away from non relavent points.

The algorithm can be modified as below:

$\overrightarrow{q} = k_0{\overrightarrow{q}}+\frac{k_1}{|D_1|}\displaystyle\sum_{d_j \epsilon D_1}\overrightarrow{d_j} +\frac{k_2}{|D_2|}\displaystyle\sum_{d_j \epsilon D_2}\overrightarrow{d_j} +\frac{k_3}{|D_3|}\displaystyle\sum_{d_j \epsilon D_3}\overrightarrow{d_j} +\frac{k_4}{|D_4|}\displaystyle\sum_{d_j \epsilon D_4}\overrightarrow{d_j} +\frac{k_5}{|D_5|}\displaystyle\sum_{d_j \epsilon D_5}\overrightarrow{d_j} $

* $k_0$ is the constant to multiply with query

* $k_1, k_2, k_3, k_4, k_5 $ are the constant multiplier to each group of documents in the feedback set.

* $k_1, k_2$ will be negative numbers as they represent constant multipliers for non relavent documents and in general $k_1$ will be larger than $k_2$.

* $k_4, k_5$ will be positive numbers as they represent constant multipliers for relavent documents and in general $k_5$ will be larger than $k_4$.

* These constants can be varied to tune the performance of the relavence feedback algorithm.




A typical IR systems which retrieves relevant documents can be improvised by suggesting additional query terms to the user. This can be done at global and local level. Below are some of the techniques I included query expansion or query refinement

### Global Query Expansion Techniques




1.   **Spelling check**: A word dictionary can be built with all the known words in the collection and the words in the WordNet. This can be used to find the correct word and suggest it to the user. Correct words can be identified using methods like edit distance, Linear interpolation using trigram probabilities, named entity recognition, parts of speech tagging etc. 
2.   **Manual Thesaurus building**: A manual Thesaurus can be built by collecting the word synonyms. Also different domains (like Law, Medical) have different set of vocabulary and domain experts can build a Thesaurus with all the relevant words. Individual query terms can be compared with this Thesaurus and most relevant words can be identifying by using weighting schemes. Top N relevant words can be suggested to the user for adding it to the query.
3. **Co-occuring words**: Noun groups (one or more continous noun words) can be extracted from the entire collection to build a collection of frequently occuring pairs of words. The terms in the query are matched with this co-occuring words collection and the relevant ones are extract. Weighting schemes can be used to rank these co-occured words extracted and can be used for suggesting the user. 
4. **Identifying Symantics**: Grammer analysis can be done on the collection to extract the gramatical relations between words and can be suggested to the users.



### Automatic Query Expansion

#### Pseudo Relevance Feedback method

In this method, instead of getting the relevance feedback from the user, we assume that top N documents retrieved are relevant. Query weights can be recalculated using Rocchio's menthods using these top N documents and the new set of relevant documents are extracted using the recalculated query weights. In this method query weights are modified but the query is not expanded. This can improve the performance of IR model by improving the precision and recall.

#### Local Context Analysis

In this method, we take the co-occuring words that is build in the global analysis. Top N co-occuring words are extracted fro the query terms and using these, top N ranked passages are retrieved (passages are used instead of documents to improve the performance and reduce processing non relevant parts of the documents). From these top N passages, top M co-occuring words are extracted and are added to the query. This improves the performance of the IR system whilse reducing the processing time.

The above methods can be used to suggest the user with additional query terms and sometimes automatically add query terms to the search.

## Limitations of Vector Space Model

In the term frequency, term-term correlation and other techniques for calculating the similarity, we have made some assumptions or ignored some of the parameters which gives more information about information needs. Some of them are stated below:

* The basic assumption we made in using the term frequency, correlations is that the terms are independent. In both Vector Space model and Probabilistic model, the underlying assumption is that the terms are independent. However this assumption is not true as group of words in a document are dependent on the context of the passage or document.
* Order of the words are not considered. The query words and document words are considered individually and the IR model is designed ignoring the sequence of words. 

Some of the below methods can be used to build models which capture above mentioned points

### Word Similarity



1.   As a first step, we extract the words to be compared. We usually extract all the Nouns and Verbs.
2.   We then calculated the distance between each meaning in the synset of the two words.
3. The shortest path distance between the two words is calculated and meaning is captured
4. Related Hyponyms are extracted to give additional query terms based on the context of the query 



### Sentence Similarity


* Word similarity along with the domain specific words dictionary, we can calculate the sentence similarity. Sementic vectors are build based on the above synsets and hyponyms and then the similarity between the sentences are calculated which captures the semantic of the sentence rather than term frequencies.
* In traditional IR systems, because the terms are assumed as independent, a sentence with same words and different are is given same similarity score. The **word order similarity** can be used in this case to improve th performance of the IR model.


### Grouping of Documents

One of the drawbacks of the traditional IR model is that it considers all the documents are in a single cluster. This can provide challenges like query drifting away by automatic query expantion. One of the methods to improve in this aspect is to group the documents into different clusters and perform the search / extract the relavent documents based on the cluster the user is targetting. K-Means clustering technique can be used for creating the clusters.